In [ ]:
# import statements
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import re
import folium
from IPython.display import clear_output

# The method is to get consumer from kafka
def connect_consumer(topic):
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=20000, 
                                   bootstrap_servers=['127.0.0.1:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print(str(ex))
    finally:
        return _consumer

In [ ]:
# The topic name is used to get data from kafka
topic = 'sensor_loc'

# Get consumer
consumer = connect_consumer(topic)
# Use folium draw Melbourne Map
m = folium.Map(location=[-37.814, 144.966], control_scale=True, zoom_start=15)
# Sensor list is to store exceed 2000 sensor information
sensor = []
# dic is to store key and sensor information. Key is each day. The first  is '2020-12-01'
dic = {'2020-12-01':sensor}
# pre_date to store the pre_date
pre_date = '2020-12-01'

for message in consumer:
    # Decode message key and value
    key = message.key.decode('utf-8')
    data = message.value.decode('utf-8')
    # Use re to split value and get data
    data = re.split('[,:]+',data)
    sensor_id = data[1]
    date = data[3]
    lat = data[5]
    long = data[7].split('}')[0]
    # This if statement is to judge if the date is new date. If it is new date, print the previous day map.
    if date not in dic.keys():
        # Store the pre_date all sensor information
        dic[pre_date] = sensor
        # Display the map date and number of sensor
        print('Date:  '+ str(pre_date))
        print('Number of next day Sensor prediction exceed 2000:' + str(len(sensor)))
        # Add the sensor in the map.
        for x in sensor:
            folium.Marker(
                location=[x['lat'], x['long']],
                popup="ID: " +x['sensor_id'] +"\n" +"Date: "+ pre_date
            ).add_to(m)
        # display map
        display(m)
        # clear_output
        clear_output(wait=True)
        # Change the pre_date
        pre_date = date
        # Create a new map, add the date in dic.keys and empty the sensor list.
        m = folium.Map(location=[-37.814, 144.966], control_scale=True, zoom_start=15)
        dic[date] = []
        sensor = []
    else:
        # If the date is exist, just add the information
        sensor.append({'sensor_id':sensor_id, 'lat': lat, 'long': long})